In [1]:
pip install ktrain==0.19.3

     |████████████████████████████████| 25.3MB 134kB/s 
     |████████████████████████████████| 421.8MB 28kB/s 
     |████████████████████████████████| 6.7MB 46.1MB/s 
     |████████████████████████████████| 983kB 14.5MB/s 
     |████████████████████████████████| 245kB 53.3MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 1.3MB 44.4MB/s 
     |████████████████████████████████| 471kB 54.5MB/s 
     |████████████████████████████████| 327kB 58.7MB/s 
     |████████████████████████████████| 3.9MB 44.0MB/s 
     |████████████████████████████████| 450kB 58.2MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 1.1MB 51.0MB/s 
     |████████████████████████████████| 890kB 49.0MB/s 
     |████████████████████████████████| 2.9MB 47.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.19.3-cp36-none-any.whl size=25267700 sha256=40ead2b1843355ae7ef3d8d049746d9008d610b0b06af9fb3d7372d1c60632d4
  Stored 

In [4]:
import ktrain
from ktrain import text
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
data = pd.read_csv("/content/train.csv")

In [6]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
data = data.dropna()

In [9]:
x=[]
y=[]
for i in list(data['text']):
    x.append(str(i))
for i in list(data['label']):
    y.append(i)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [65]:
txt=text.Transformer('bert-base-uncased', classes=[0,1])

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:399: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


In [66]:
training=txt.preprocess_train(x_train,y_train)
validation=txt.preprocess_test(x_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 807
	95percentile : 1931
	99percentile : 4182


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 794
	95percentile : 1872
	99percentile : 3952


In [67]:
model=txt.get_classifier()
learner=ktrain.get_learner(model,train_data=training,val_data=validation,batch_size=3)

In [68]:
learner.autofit(3e-5,1)



begin training using triangular learning rate policy with max lr of 3e-05...
Train for 4267 steps, validate for 172 steps
4267/4267 [==============================] - 573s 134ms/step - loss: 0.1162 - accuracy: 0.9578 - val_loss: 0.0292 - val_accuracy: 0.9905


In [69]:
predictor=ktrain.get_predictor(learner.model, txt)

In [70]:
test_data = pd.read_csv("/content/test.csv")

In [71]:
test_data.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [72]:
test_data.fillna('fake fake fake',inplace=True)

In [73]:
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,fake fake fake,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [74]:
test_message = test_data.copy()
test_message.reset_index(inplace=True)
test_message.head(5)

,index,id,title,author,text
0,0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,1,20801,Russian warships ready to strike terrorists ne...,fake fake fake,Russian warships ready to strike terrorists ne...
2,2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [75]:
test_pred = predictor.predict(list(test_message["text"]))

In [76]:
type(test_pred[0])

int

In [77]:
submission = pd.DataFrame({'id':test_message["id"], 'label':test_pred})
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [79]:
submission.to_csv('ktrain_submission.csv',index=False)